In [5]:
import itertools

# Base probabilities
PROBS = {
    "gene": {
        2: 0.01,
        1: 0.03,
        0: 0.96
    },
    "trait": {
        2: {True: 0.65, False: 0.35},
        1: {True: 0.56, False: 0.44},
        0: {True: 0.01, False: 0.99}
    }
}

def powerset(s):
    """
    Return a list of all possible subsets of set s.
    """
    s = list(s)
    return [
        set(combo) for combo in itertools.chain.from_iterable(
            itertools.combinations(s, r) for r in range(len(s)+1)
        )
    ]

def joint_probability(people, one_gene, two_genes, have_trait):
    """
    Compute joint probability of a given configuration.
    """
    probability = 1

    for person in people:
        genes = (2 if person in two_genes else
                 1 if person in one_gene else 0)

        mother = people[person]["mother"]
        father = people[person]["father"]

        # If no parents, use unconditional gene probability
        if mother is None and father is None:
            probability *= PROBS["gene"][genes]
        else:
            # Probability parent passes gene
            def pass_prob(parent):
                if parent in two_genes:
                    return 0.99
                elif parent in one_gene:
                    return 0.5
                else:
                    return 0.01

            mom_prob = pass_prob(mother)
            dad_prob = pass_prob(father)

            if genes == 2:
                probability *= mom_prob * dad_prob
            elif genes == 1:
                probability *= mom_prob * (1 - dad_prob) + (1 - mom_prob) * dad_prob
            else:  # 0 genes
                probability *= (1 - mom_prob) * (1 - dad_prob)

        # Trait probability
        probability *= PROBS["trait"][genes][person in have_trait]

    return probability

def update(probabilities, one_gene, two_genes, have_trait, p):
    """
    Add joint probability p to totals for each person.
    """
    for person in probabilities:
        genes = (2 if person in two_genes else
                 1 if person in one_gene else 0)
        probabilities[person]["gene"][genes] += p
        probabilities[person]["trait"][person in have_trait] += p

def normalize(probabilities):
    """
    Normalize so probabilities sum to 1.
    """
    for person in probabilities:
        # Normalize gene
        total_gene = sum(probabilities[person]["gene"].values())
        for g in probabilities[person]["gene"]:
            probabilities[person]["gene"][g] /= total_gene

        # Normalize trait
        total_trait = sum(probabilities[person]["trait"].values())
        for t in probabilities[person]["trait"]:
            probabilities[person]["trait"][t] /= total_trait

# --------------------
# Example: Small Family
# --------------------
people = {
    "Alice": {"mother": None, "father": None},
    "Bob": {"mother": None, "father": None},
    "Charlie": {"mother": "Alice", "father": "Bob"}
}

# Initialize probabilities
probabilities = {
    person: {
        "gene": {2: 0, 1: 0, 0: 0},
        "trait": {True: 0, False: 0}
    }
    for person in people
}

names = set(people)
for have_trait in powerset(names):
    for one_gene in powerset(names - have_trait):
        for two_genes in powerset(names - one_gene - have_trait):
            p = joint_probability(people, one_gene, two_genes, have_trait)
            update(probabilities, one_gene, two_genes, have_trait, p)

normalize(probabilities)

# Print results
import pprint
pprint.pprint(probabilities)

{'Alice': {'gene': {0: 0.9885108431404465,
                    1: 0.009862819548130715,
                    2: 0.001626337311422801},
           'trait': {False: 0.9901148915685954, True: 0.009885108431404468}},
 'Bob': {'gene': {0: 0.9885108431404465,
                  1: 0.009862819548130715,
                  2: 0.0016263373114228008},
         'trait': {False: 0.9901148915685954, True: 0.00988510843140447}},
 'Charlie': {'gene': {0: 0.9820965063146384,
                      1: 0.017758364595940684,
                      2: 0.00014512908942096103},
             'trait': {False: 0.9901790349368536, True: 0.009820965063146387}}}
